In [1]:
pip install earthengine-api

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import ee
ee.Authenticate()
ee.Initialize()

In [3]:
# Define the bounding box for Sri Lanka(Area of Interest)
min_lon = 79.6  # Minimum longitude (west)
min_lat = 5.9   # Minimum latitude (south)
max_lon = 81.9  # Maximum longitude (east)
max_lat = 9.8   # Maximum latitude (north

In [4]:
sst_dataset = ee.ImageCollection("NOAA/CDR/OISST/V2_1").select("sst")

# Create a rectangular region for Sri Lanka
region = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

# Filter data by date and region
filtered_sst = sst_dataset.filterDate('2024-01-01', '2024-12-31').filterBounds(region)

In [5]:
print("SST Data: ", filtered_sst.size().getInfo())

SST Data:  365


In [6]:
print(filtered_sst.getInfo())

{'type': 'ImageCollection', 'bands': [], 'version': 1737494804064458, 'id': 'NOAA/CDR/OISST/V2_1', 'properties': {'date_range': [368150400000, 1647734400000], 'period_mapping': [368150400000, 1647734400000], 'period': 1, 'type_name': 'ImageCollection', 'keywords': ['avhrr', 'cdr', 'daily', 'ice', 'noaa', 'ocean', 'oisst', 'real-time', 'sst', 'temperature'], 'system:is_global': 1, 'thumb': 'https://mw1.google.com/ges/dd/images/OISST_thumb.png', 'description': '<p>The NOAA 1/4 degree daily Optimum Interpolation Sea Surface Temperature\n(OISST) provides complete ocean temperature fields constructed by combining\nbias-adjusted observations from different platforms (satellite, ships,\nbuoys) on a regular global grid, with gaps filled in by interpolation.\nSatellite data from the Advanced Very High Resolution Radiometer (AVHRR)\nprovides the main input which permits the high temporal-spatial coverage\nbeginning in late 1981 to the present.</p><p>The OISST dataset has a single day&#39;s data 

In [7]:
# Function to calculate mean SST for each image in the collection
def add_mean_sst(image):
    # Calculate the mean SST value within the region
    mean_sst = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=5000  # Adjust based on your data resolution
    ).get('sst')

    # Get the date of the image
    date = image.date().format('YYYY-MM-dd')

    # Create a feature with date and mean SST
    feature = ee.Feature(None, {
        'date': date,
        'mean_sst': mean_sst
    })
    return feature

# Apply the function to each image in the filtered collection
mean_sst_features = filtered_sst.map(add_mean_sst)

In [8]:

# Export the FeatureCollection with mean SST values to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=mean_sst_features,
    description='Mean_SST_Values_2024',
    fileFormat='CSV'
)

# Start the export task
task.start()